# Method 2: Model Selection and Averaging

Set up the environments

In [4]:
import os

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

Load data

In [5]:
db=pd.read_csv("data/new_database.csv")
form=pd.read_csv("data/evaluation_public.csv")
submit=pd.read_csv("data/submit_example.csv")
search=pd.read_csv("data/train_search_data.csv")
reply=pd.read_csv("data/train_user_reply_data.csv")
province_list=list(set(db.province))
model_list=list(set(db.model))

Initialize the score

In [6]:
NRMSE=0
NRMSE_e=0

Watching the progress

In [7]:
count=0
count_10=10

Iterate by province and car model and by model selection

In [16]:
for prov in province_list:
    for mode in model_list:
        cscore=100000000 #+\infty
        grid_size=3
        for w_1 in range(grid_size):
            for w_2 in range(grid_size-int(w_1)):
                for w_3 in range(grid_size-int(w_1+w_2)):
                    if w_1+w_2+w_3==0:
                        break
                    
                    # brute-force grid seach for model selection
                    model_1_options=2
                    model_2_options=2
                    for m_11 in range(model_1_options):
                        for m_2 in range(model_2_options):
                            m_21=m_2+2
                            for m_3 in range(2): 
                                m_31=m_3+4
                                # brute-force grid search weights to maximize score
                                ##########################
                                # read data
                                db_selected=db[(db.province==prov) & (db.model==mode)]
                                series_original=pd.Series(db_selected.salesVolume)
                                series=[]
                                for i in series_original.index:
                                    series.append(series_original[i])
                                series_original=pd.Series(db_selected.popularity)
                                popu=[]
                                for i in series_original.index:
                                    popu.append(series_original[i])
                                series_original=pd.Series(db_selected.carCommentVolum)
                                comm=[]
                                for i in series_original.index:
                                    comm.append(series_original[i])
                                series_original=pd.Series(db_selected.newsReplyVolum)
                                repl=[]
                                for i in series_original.index:
                                    repl.append(series_original[i])
                                
                                #################################
                                # Choose parameters
                                size=12 # 11+1(index)
                                data = np.zeros((17,size))
                                data[0,:]=range(size)
                                for i in range(16): # 24-12+4
                                    data[i+1][0]=i+1 #index
                                    # Lagging terms -- Better than log, reciprocal, square root, power 0.2,0.7 after testing 
                                    data[i+1][2]=series[i]**0.5
                                    data[i+1][3]=series[i+6]**0.5
                                    
                                    # Differentiations
                                    data[i+1][4]=series[i+8]-series[i+7]
                                    data[i+1][5]=series[i+1]-series[i]
                                    
                                    # Exterior variables
                                    data[i+1][6]=100*np.log(popu[i]+0.000000000001)
                                    data[i+1][7]=100*np.log(comm[i]+0.000000000001)
                                
                                for i in range(12):
                                    data[i+1][1]=series[i+12] # Y
                                for i in range(4):
                                    data[i+1+12][1]=float("NaN") # No data
                                    
                                    
                                #############################################
                                # model 1: lagging terms
                                #############################################
                                X_train=np.zeros((8,1))
                                X_train[:,0]=data[1:9,m_11+2]
                                Y_train=data[1:9,1]
                                
                                ################################
                                # Regression
                                model = LinearRegression()
                                model.fit(X_train,Y_train)
                                a  = model.intercept_# intercept
                                b = model.coef_# coeffitient
                                
                                # Calculate score to be optimized
                                X_test=np.zeros((2,1))
                                X_test[:,0]=data[[9,10],m_11+2]
                                Y_1=np.dot(X_test,b)+a
                                
                                # Calculate score on test set
                                X_exam=np.zeros((2,1))
                                X_exam[:,0]=data[[11,12],m_11+2]
                                Y_1_e=np.dot(X_exam,b)+a
                                
                                # Results to be submitted
                                X_submit=np.zeros((4,1))
                                X_submit[:,0]=data[[13,14,15,16],m_11+2]
                                Y_1_submit=np.dot(X_submit,b)+a
                                
                                #############################################
                                # end of model 1
                                #############################################
                                
                                #############################################
                                # model 2: 1st order difference
                                #############################################
                                X_train=np.zeros((8,1))
                                X_train[:,0]=data[1:9,m_21+2]
                                Y_train=data[1:9,1]
                                
                                ################################
                                # Regression
                                model = LinearRegression()
                                model.fit(X_train,Y_train)
                                a  = model.intercept_# intercept
                                b = model.coef_# coefficient
                                
                                # Calculate score to be optimized
                                X_test=np.zeros((2,1))
                                X_test[:,0]=data[[9,10],m_21+2]
                                Y_2=np.dot(X_test,b)+a
                                
                                # Calculate score on test set
                                X_exam=np.zeros((2,1))
                                X_exam[:,0]=data[[11,12],m_21+2]
                                Y_2_e=np.dot(X_exam,b)+a
                                
                                # Results to be submitted
                                X_submit=np.zeros((4,1))
                                X_submit[:,0]=data[[13,14,15,16],m_21+2]
                                Y_2_submit=np.dot(X_submit,b)+a
                                
                                #############################################
                                # end of model 2
                                #############################################
                                
                                #############################################
                                # model 3: exterior variables
                                #############################################                                
                                X_train=np.zeros((8,1))
                                X_train[:,0]=data[1:9,m_31]
                                Y_train=data[1:9,1]
                                
                                ################################
                                # Regression
                                model = LinearRegression()
                                model.fit(X_train,Y_train)
                                a  = model.intercept_# intercept
                                b = model.coef_# coefficient
                                
                                # Calculate score to be optimized
                                X_test=np.zeros((2,1))
                                X_test[:,0]=data[[9,10],m_31]
                                Y_3=np.dot(X_test,b)+a
                                
                                # Calculate score on test set
                                X_exam=np.zeros((2,1))
                                X_exam[:,0]=data[[11,12],m_31]
                                Y_3_e=np.dot(X_exam,b)+a
                                
                                # Results to be submitted
                                X_submit=np.zeros((4,1))
                                X_submit[:,0]=data[[13,14,15,16],m_31]
                                Y_3_submit=np.dot(X_submit,b)+a
                                #############################################
                                # end of model 3
                                #############################################
                                
                                #############################################
                                # model averaging
                                #############################################
                                #weights
                                #w_3=0
                                w_1=(w_1)/(w_1+w_2+w_3)
                                w_2=(w_2)/(w_1+w_2+w_3)
                                w_3=(w_3)/(w_1+w_2+w_3)
                                Y_hat=w_1*Y_1+w_2*Y_2+w_3*Y_3
                                
                                #############################################
                                # end of model averaging
                                #############################################
                                
                                ################################
                                # Calculate score to be optimized
                                Y_test=data[[9,10],1]  # optimize set [1,2,9,10]
                                sum=0
                                mean=0
            
                                for i in range(2):
                                    sum=sum+(Y_hat[i]-Y_test[i])*(Y_hat[i]-Y_test[i])
                                    mean=mean+series[i]/2
                                if np.sqrt(sum/2)/mean<cscore:
                                    cscore=np.sqrt(sum/2)/mean
                                    weight_1=w_1
                                    weight_2=w_2
                                    weight_3=w_3

        Y_hat=weight_1*Y_1+weight_2*Y_2+weight_3*Y_3
        Y_hat_exam=weight_1*Y_1_e+weight_2*Y_2_e+weight_3*Y_3_e
        # Calculate score to be optimized
        sum=0
        mean=0
        for i in range(2):
            sum=sum+(Y_hat[i]-Y_test[i])*(Y_hat[i]-Y_test[i])
            mean=mean+series[i]/2
        NRMSE=NRMSE+np.sqrt(sum/2)/mean
        
        # Calculate score on test set
        Y_exam=data[[11,12],1]
        sum_e=0
        mean_e=0
        
        for i in range(2):
            sum_e=sum_e+(Y_hat_exam[i]-Y_exam[i])*(Y_hat_exam[i]-Y_exam[i])
            mean_e=mean_e+series[i]/2
        NRMSE_e=NRMSE_e+np.sqrt(sum_e/2)/mean_e
        
        # Data to be submitted
        Y_hat_submit=weight_1*Y_1_submit+weight_2*Y_2_submit+weight_3*Y_3_submit

        for i in range(4):
            idx=form[(form.province==prov) & (form.model==mode) & (form.regMonth==i+1)].index.values
            submit.loc[idx,'forecastVolum']=max(int(Y_hat_submit[i]),0)

        # watch progress
        count+=1
        print(100*count/(22*60),'% completed')

0.07575757575757576 % completed
0.15151515151515152 % completed
0.22727272727272727 % completed
0.30303030303030304 % completed
0.3787878787878788 % completed
0.45454545454545453 % completed
0.5303030303030303 % completed
0.6060606060606061 % completed
0.6818181818181818 % completed
0.7575757575757576 % completed
0.8333333333333334 % completed
0.9090909090909091 % completed
0.9848484848484849 % completed
1.0606060606060606 % completed
1.1363636363636365 % completed
1.2121212121212122 % completed
1.2878787878787878 % completed
1.3636363636363635 % completed
1.4393939393939394 % completed
1.5151515151515151 % completed
1.5909090909090908 % completed
1.6666666666666667 % completed
1.7424242424242424 % completed
1.8181818181818181 % completed
1.893939393939394 % completed
1.9696969696969697 % completed
2.0454545454545454 % completed
2.121212121212121 % completed
2.196969696969697 % completed
2.272727272727273 % completed
2.3484848484848486 % completed
2.4242424242424243 % completed
2.5 % c

20.681818181818183 % completed
20.757575757575758 % completed
20.833333333333332 % completed
20.90909090909091 % completed
20.984848484848484 % completed
21.060606060606062 % completed
21.136363636363637 % completed
21.21212121212121 % completed
21.28787878787879 % completed
21.363636363636363 % completed
21.439393939393938 % completed
21.515151515151516 % completed
21.59090909090909 % completed
21.666666666666668 % completed
21.742424242424242 % completed
21.818181818181817 % completed
21.893939393939394 % completed
21.96969696969697 % completed
22.045454545454547 % completed
22.12121212121212 % completed
22.196969696969695 % completed
22.272727272727273 % completed
22.348484848484848 % completed
22.424242424242426 % completed
22.5 % completed
22.575757575757574 % completed
22.651515151515152 % completed
22.727272727272727 % completed
22.803030303030305 % completed
22.87878787878788 % completed
22.954545454545453 % completed
23.03030303030303 % completed
23.106060606060606 % completed

41.28787878787879 % completed
41.36363636363637 % completed
41.43939393939394 % completed
41.515151515151516 % completed
41.59090909090909 % completed
41.666666666666664 % completed
41.74242424242424 % completed
41.81818181818182 % completed
41.89393939393939 % completed
41.96969696969697 % completed
42.04545454545455 % completed
42.121212121212125 % completed
42.196969696969695 % completed
42.27272727272727 % completed
42.34848484848485 % completed
42.42424242424242 % completed
42.5 % completed
42.57575757575758 % completed
42.65151515151515 % completed
42.72727272727273 % completed
42.803030303030305 % completed
42.878787878787875 % completed
42.95454545454545 % completed
43.03030303030303 % completed
43.10606060606061 % completed
43.18181818181818 % completed
43.25757575757576 % completed
43.333333333333336 % completed
43.40909090909091 % completed
43.484848484848484 % completed
43.56060606060606 % completed
43.63636363636363 % completed
43.71212121212121 % completed
43.787878787878

62.121212121212125 % completed
62.196969696969695 % completed
62.27272727272727 % completed
62.34848484848485 % completed
62.42424242424242 % completed
62.5 % completed
62.57575757575758 % completed
62.65151515151515 % completed
62.72727272727273 % completed
62.803030303030305 % completed
62.878787878787875 % completed
62.95454545454545 % completed
63.03030303030303 % completed
63.10606060606061 % completed
63.18181818181818 % completed
63.25757575757576 % completed
63.333333333333336 % completed
63.40909090909091 % completed
63.484848484848484 % completed
63.56060606060606 % completed
63.63636363636363 % completed
63.71212121212121 % completed
63.78787878787879 % completed
63.86363636363637 % completed
63.93939393939394 % completed
64.01515151515152 % completed
64.0909090909091 % completed
64.16666666666667 % completed
64.24242424242425 % completed
64.31818181818181 % completed
64.39393939393939 % completed
64.46969696969697 % completed
64.54545454545455 % completed
64.62121212121212 

83.18181818181819 % completed
83.25757575757575 % completed
83.33333333333333 % completed
83.4090909090909 % completed
83.48484848484848 % completed
83.56060606060606 % completed
83.63636363636364 % completed
83.71212121212122 % completed
83.78787878787878 % completed
83.86363636363636 % completed
83.93939393939394 % completed
84.01515151515152 % completed
84.0909090909091 % completed
84.16666666666667 % completed
84.24242424242425 % completed
84.31818181818181 % completed
84.39393939393939 % completed
84.46969696969697 % completed
84.54545454545455 % completed
84.62121212121212 % completed
84.6969696969697 % completed
84.77272727272727 % completed
84.84848484848484 % completed
84.92424242424242 % completed
85.0 % completed
85.07575757575758 % completed
85.15151515151516 % completed
85.22727272727273 % completed
85.3030303030303 % completed
85.37878787878788 % completed
85.45454545454545 % completed
85.53030303030303 % completed
85.60606060606061 % completed
85.68181818181819 % complet

Submit results

In [17]:
submit.to_csv('results/submit_results_model_sel_avg.csv',sep=',',encoding='utf-8',index=None)

Calculate score on test set

In [18]:
Score=1-NRMSE/(len(province_list)*len(model_list))
Score

Evaluation=1-NRMSE_e/(len(province_list)*len(model_list))
print('Score on test set::',Evaluation)

Score on test set:: 0.5529655954707162
